# Generating artificial data

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import os


In [4]:
import numpy as np
import os

np.random.seed(1234)  # For reproducibility
n_series = 4000   # Number of time series per class
n_points = 500    # Number of data points in each time series

#output_dir = 'time_series_data'
#os.makedirs(output_dir, exist_ok=True)

output_dir = os.path.expanduser("~/timeseries_data")  # Creates the directory in your home folder
os.makedirs(output_dir, exist_ok=True)


### AR, MA and ARMA Model

In [6]:
# Function to generate AR, MA, or ARMA data with optional trend and seasonality
def generate_time_series(model_type, order, n_points, trend_strength=0.1, seasonality_amplitude=0.5, seasonality_period=50, include_trend=True):
   if model_type == 'AR':
       params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.r_[1, -params]
       ma = np.array([1])
   elif model_type == 'MA':
       params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.array([1])
       ma = np.r_[1, params]
   elif model_type == 'ARMA':
       ar_params = np.random.uniform(-0.5, 0.5, size=order)
       ma_params = np.random.uniform(-0.5, 0.5, size=order)
       ar = np.r_[1, -ar_params]
       ma = np.r_[1, ma_params]
   else:
       raise ValueError("Invalid model type. Use 'AR', 'MA', or 'ARMA'.")
   
   
      # Generate the process
   process = sm.tsa.ArmaProcess(ar, ma)
   data = process.generate_sample(nsample=n_points)
   if include_trend:
       trend = np.linspace(0, trend_strength * n_points, n_points)
       seasonality = seasonality_amplitude * np.sin(2 * np.pi * np.arange(n_points) / seasonality_period)
       data += trend + seasonality
   else:
       seasonality = seasonality_amplitude * np.sin(2 * np.pi * np.arange(n_points) / seasonality_period)
       data += seasonality
   return data

### Models for orders 1-3 with and without trend (Kernel crashed after 14 classes)

In [15]:
# Loop to generate and save time series plots for each class
model_types = ['AR', 'MA', 'ARMA']
orders = [1, 2, 3]
for model_type in model_types:
   for order in orders:
       for include_trend in [True, False]:
           class_label = f'{model_type}_{order}_with_trend' if include_trend else f'{model_type}_{order}_without_trend'
           class_dir = os.path.join(output_dir, class_label)
           os.makedirs(class_dir, exist_ok=True)
           for i in range(n_series):
               data = generate_time_series(model_type, order, n_points, include_trend=include_trend)
               # Plotting the time series
               plt.figure(figsize=(8, 4))
               plt.plot(data)
               plt.axis('off')  # Turn off axes for a clean image
               plt.savefig(os.path.join(class_dir, f'Series_{i+1}.png'), bbox_inches='tight', pad_inches=0)
               plt.close()
print("Time series generation completed. Time series are saved in the 'time_series_data' directory.")

: 

### Generating image for remaining classes

In [7]:
# Adjusted classes for ARMA 2 and 3 with and without trend
model_type = 'ARMA'
orders = [2, 3]
for order in orders:
    for include_trend in [True, False]:
        class_label = f'{model_type}_{order}_with_trend' if include_trend else f'{model_type}_{order}_without_trend'
        class_dir = os.path.join(output_dir, class_label)
        os.makedirs(class_dir, exist_ok=True)
        for i in range(n_series):
            data = generate_time_series(model_type, order, n_points, include_trend=include_trend)
            # Plotting the time series
            plt.figure(figsize=(8, 4))
            plt.plot(data)
            plt.axis('off')  # Turn off axes for a clean image
            plt.savefig(os.path.join(class_dir, f'Series_{i+1}.png'), bbox_inches='tight', pad_inches=0)
            plt.close()

# Code to check the number of images in each folder
folder_status = {}
for folder_name in os.listdir(output_dir):
    folder_path = os.path.join(output_dir, folder_name)
    if os.path.isdir(folder_path):
        folder_status[folder_name] = len(os.listdir(folder_path))

# Display folder status
for class_label, image_count in folder_status.items():
    print(f"{class_label}: {image_count} images")

ARMA_1_without_trend: 4000 images
ARMA_2_with_trend: 4000 images
MA_3_with_trend: 4000 images
ARMA_2_without_trend: 4000 images
AR_3_without_trend: 4000 images
MA_1_without_trend: 4000 images
MA_1_with_trend: 4000 images
AR_2_with_trend: 4000 images
MA_2_without_trend: 4000 images
AR_1_with_trend: 4000 images
MA_2_with_trend: 4000 images
MA_3_without_trend: 4000 images
ARMA_3_with_trend: 4000 images
AR_3_with_trend: 4000 images
AR_2_without_trend: 4000 images
ARMA_3_without_trend: 4000 images
AR_1_without_trend: 4000 images
ARMA_1_with_trend: 4000 images


# Loading the data

In [2]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import SeparableConv2D
from sklearn.utils import shuffle

In [3]:
# Parameters
# Global parameters
image_size = (64, 64)  # Resize all images to this size
main_dir = os.path.expanduser("~/timeseries_data")  # Path to your main directory with class subfolders


def load_data(main_dir, image_size):
    data = []
    labels = []
    classes = sorted([cls for cls in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, cls))])  # Filter directories only
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}  # Mapping class names to indices

    for cls in classes:
        class_dir = os.path.join(main_dir, cls)
        for img_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_file)
            if img_file.endswith(('.png', '.jpg', '.jpeg')):  # Ensure it's an image file
                img = Image.open(img_path).convert('RGB')  # Convert to RGB
                img = img.resize(image_size)  # Resize image
                data.append(np.array(img))
                labels.append(class_to_idx[cls])

    return np.array(data), np.array(labels), classes




In [4]:
# Generate a mapping for classes
classes = sorted([cls for cls in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, cls))])
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}  # {'AR_1_with_trend': 0, 'MA_1_without_trend': 1, ...}
print(f"Class to index mapping: {class_to_idx}")


Class to index mapping: {'ARMA_1_with_trend': 0, 'ARMA_1_without_trend': 1, 'ARMA_2_with_trend': 2, 'ARMA_2_without_trend': 3, 'ARMA_3_with_trend': 4, 'ARMA_3_without_trend': 5, 'AR_1_with_trend': 6, 'AR_1_without_trend': 7, 'AR_2_with_trend': 8, 'AR_2_without_trend': 9, 'AR_3_with_trend': 10, 'AR_3_without_trend': 11, 'MA_1_with_trend': 12, 'MA_1_without_trend': 13, 'MA_2_with_trend': 14, 'MA_2_without_trend': 15, 'MA_3_with_trend': 16, 'MA_3_without_trend': 17}


In [5]:
# Load the dataset
print("Loading dataset...")
data, labels, classes = load_data(main_dir, image_size)

# Normalize data
data = data / 255.0  # Normalize pixel values to [0, 1]

# Print class information
print(f"Classes: {classes}")
print(f"Number of classes: {len(classes)}")


Loading dataset...
Classes: ['ARMA_1_with_trend', 'ARMA_1_without_trend', 'ARMA_2_with_trend', 'ARMA_2_without_trend', 'ARMA_3_with_trend', 'ARMA_3_without_trend', 'AR_1_with_trend', 'AR_1_without_trend', 'AR_2_with_trend', 'AR_2_without_trend', 'AR_3_with_trend', 'AR_3_without_trend', 'MA_1_with_trend', 'MA_1_without_trend', 'MA_2_with_trend', 'MA_2_without_trend', 'MA_3_with_trend', 'MA_3_without_trend']
Number of classes: 18


In [6]:
from sklearn.model_selection import train_test_split
import numpy as np

# Subset of classes to start with (e.g., 6 representative classes)
subset_classes = ['AR_1_with_trend', 'AR_2_without_trend', 
                  'MA_1_with_trend', 'MA_2_without_trend', 
                  'ARMA_1_with_trend', 'ARMA_2_without_trend']

# Map subset classes to indices
subset_indices = [class_to_idx[cls] for cls in subset_classes]

# Filter data and labels for the subset
subset_mask = np.isin(labels, subset_indices)
data_subset = data[subset_mask]
labels_subset = labels[subset_mask]

# Reindex labels for the subset
labels_subset = np.array([subset_indices.index(lbl) for lbl in labels_subset])

# Step 1: Split the subset into train (70%) and temp (30%) using stratified sampling
X_train, X_temp, y_train, y_temp = train_test_split(
    data_subset, labels_subset, test_size=0.3, stratify=labels_subset, random_state=42
)

# Step 2: Split temp into validation (20%) and test (10%) using stratified sampling
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=1/3, stratify=y_temp, random_state=42
)  # 1/3 of temp goes to the test set

# Print dataset sizes to verify the split
print(f"Training set: {X_train.shape}, Labels: {y_train.shape}")
print(f"Validation set: {X_val.shape}, Labels: {y_val.shape}")
print(f"Test set: {X_test.shape}, Labels: {y_test.shape}")


Training set: (16800, 64, 64, 3), Labels: (16800,)
Validation set: (4800, 64, 64, 3), Labels: (4800,)
Test set: (2400, 64, 64, 3), Labels: (2400,)


# --- CNN extractor with SVM 

In [17]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import numpy as np

# CNN Feature Extractor: ResNet50
base_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global pooling to reduce dimensions
feature_extractor_model = Model(inputs=base_model.input, outputs=x)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False
    
# Function to extract features using the CNN
def extract_features(data, model, batch_size=32):
    num_samples = data.shape[0]
    features = []
    for i in range(0, num_samples, batch_size):
        batch_data = data[i:i+batch_size]
        # Resize batch to 224x224 dynamically
        batch_data = tf.image.resize(batch_data, [224, 224])
        batch_features = model.predict(batch_data)
        features.append(batch_features)
    return np.vstack(features)


# Extract features for training, validation, and testing
print("Extracting features...")
train_features = extract_features(X_train, feature_extractor_model)
val_features = extract_features(X_val, feature_extractor_model)
test_features = extract_features(X_test, feature_extractor_model)

print(f"Train features shape: {train_features.shape}")
print(f"Validation features shape: {val_features.shape}")
print(f"Test features shape: {test_features.shape}")

# SVM Classifier
print("Training the SVM...")
svm = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1.0, gamma='scale', probability=True))
svm.fit(train_features, y_train)

# Validate the SVM
val_predictions = svm.predict(val_features)
print("Validation Classification Report:")
print(classification_report(y_val, val_predictions, target_names=subset_classes))

# Test the SVM
test_predictions = svm.predict(test_features)
print("Test Classification Report:")
print(classification_report(y_test, test_predictions, target_names=subset_classes))


Extracting features...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 711ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 711ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 707ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 714ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 712ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 706ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 707ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 701ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 715ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 710ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 696ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 701ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 694ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 704ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 781ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 803ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 711ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 707ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 715ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 708ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 788ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 909ms/step
1/1 ━

In [22]:
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")


X_train shape: (16800, 64, 64, 3)
X_val shape: (4800, 64, 64, 3)
X_test shape: (2400, 64, 64, 3)


# --- Pretrained ViT with 6 classes 

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2

# Parameters
IMAGE_SIZE = (224, 224)  # Image size for MobileNetV2
NUM_CLASSES = 18         # Number of classes
BATCH_SIZE = 32
EPOCHS = 30

# Load MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(*IMAGE_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')

# Add additional classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduces the dimensions of the feature output
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.6)(x)  # Dropout to prevent overfitting
output = Dense(NUM_CLASSES, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model (for initial training)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model structure
model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    X_train_resized, y_train,
    validation_data=(X_val_resized, y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr]
)

# Unfreeze the base model and perform fine-tuning
for layer in base_model.layers:
    layer.trainable = True

# Fine-tuning with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train fine-tuning
history_fine = model.fit(
    X_train_resized, y_train,
    validation_data=(X_val_resized, y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_resized, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,424,274 (9.25 MB)

 Trainable params: 166,290 (649.57 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 119s 222ms/step - accuracy: 0.3276 - loss: 2.2576 - val_accuracy: 0.3348 - val_loss: 1.1962 - learning_rate: 0.0010
Epoch 2/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 116s 221ms/step - accuracy: 0.3404 - loss: 1.1846 - val_accuracy: 0.3477 - val_loss: 1.1223 - learning_rate: 0.0010
Epoch 3/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 115s 220ms/step - accuracy: 0.3393 - loss: 1.1302 - val_accuracy: 0.3333 - val_loss: 1.1169 - learning_rate: 0.0010
Epoch 4/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 114s 217ms/step - accuracy: 0.3427 - loss: 1.1222 - val_accuracy: 0.3458 - val_loss: 1.1076 - learning_rate: 0.0010
Epoch 5/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 114s 218ms/step - accuracy: 0.3393 - loss: 1.1196 - val_accuracy: 0.3338 - val_loss: 1.1055 - learning_rate: 0.0010
Epoch 6/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 115s 218ms/step - accuracy: 0.3430 - loss: 1.1175 - val_accuracy: 0.3542 - val_loss: 1.1051 - learning_rate: 0.0010
Epoch 7/30
525/525 ━━━━━━━━━━━━━━━━━━━━ 115s 219ms/step - accura